#### 라이브러리 불러오기

In [20]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

from datetime import datetime

#### 데이터 불러오기

In [3]:
(train_x, train_y), (test_x, test_y) = cifar10.load_data()

In [7]:
_max_value = train_x.max().astype(np.float32)
train_x, test_x = train_x / _max_value, test_x / _max_value

In [8]:
CLASS_NAMES = [
    'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
]

In [9]:
BATCH_SIZE = 32

##### TF Dataset 만들기

In [10]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))\
    .shuffle(train_x.shape[0]).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(BATCH_SIZE)

2023-08-23 21:12:24.243359: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-23 21:12:24.247605: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-23 21:12:24.247763: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

#### 모델 생성

In [12]:
class Cifar10CNNModel(tf.keras.Model):
    
    def __init__(self, 
                 filter1, kernel_size1, pool_size1, 
                 filter2, kernel_size2, pool_size2,
                 filter3, kernel_size3, pool_size3,
                 dense1, dense2):
        super(Cifar10CNNModel, self).__init__(name='Cifar10CNNModel')
        
        self.data = Conv2D(
            filter1, kernel_size1, activation='relu', input_shape=(train_x.shape[1:]))
        self.mp1 = MaxPooling2D(pool_size1)
        self.h1 = Conv2D(filter2, kernel_size2, activation='relu')
        self.mp2 = MaxPooling2D(pool_size2)
        self.h2 = Conv2D(filter3, kernel_size3, activation='relu')
        self.mp3 = MaxPooling2D(pool_size3)
        self.f = Flatten()
        self.fc1 = Dense(dense1, activation='relu')
        self.fc2 = Dense(dense2, activation='relu')
        self.result = Dense(len(CLASS_NAMES), activation='softmax')
        
    def call(self, input):
        model = self.data(input)
        model = self.mp1(model)
        model = self.h1(model)
        model = self.mp2(model)
        model = self.h2(model)
        model = self.mp3(model)
        model = self.f(model)
        model = self.fc1(model)
        model = self.fc2(model)
        model = self.result(model)
        return model

In [15]:
@tf.function
def train_step(images, labels, model, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        _loss = loss(labels, predictions)
    gradients = tape.gradient(_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(_loss)
    train_accuracy(labels, predictions)

In [16]:
@tf.function
def test_step(images, labels, model, test_loss, test_accuracy):
    predictions = model(images, training=False)
    _loss = loss(labels, predictions)
    test_loss(_loss)
    test_accuracy(labels, predictions)

In [17]:
def create_model(
    filter1, kernel_size1, pool_size1, 
    filter2, kernel_size2, pool_size2,
    filter3, kernel_size3, pool_size3,
    dense1, dense2, lr
):
    model = Cifar10CNNModel(
        filter1, kernel_size1, pool_size1, 
        filter2, kernel_size2, pool_size2,
        filter3, kernel_size3, pool_size3,
        dense1, dense2
    )
    optimizer = tf.keras.optimizers.Adam(lr)
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy()

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    test_loss = tf.keras.metrics.Mean(name='test_loss')
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
    
    return model, optimizer, train_loss, train_accuracy, test_loss, test_accuracy

In [18]:
def training(
    filter1, kernel_size1, pool_size1, 
    filter2, kernel_size2, pool_size2,
    filter3, kernel_size3, pool_size3,
    dense1, dense2, lr, epochs
):
    model, optimizer, train_loss, train_accuracy, test_loss, test_accuracy = create_model(
        filter1, kernel_size1, pool_size1, 
        filter2, kernel_size2, pool_size2,
        filter3, kernel_size3, pool_size3,
        dense1, dense2, lr
    )
    for epoch in range(epochs):
        train_loss.reset_state()
        train_accuracy.reset_state()
        test_loss.reset_state()
        test_accuracy.reset_state()
        
        for _images, _labels in train_ds:
            train_step(_images, _labels, model, optimizer)
            
        for _images, _labels in test_ds:
            test_step(_images, _labels, model)
            
    return test_accuracy.result().numpy()

In [19]:
def target_func(
    filter1, kernel_size1, pool_size1, 
    filter2, kernel_size2, pool_size2,
    filter3, kernel_size3, pool_size3,
    dense1, dense2, lr, epochs
):
    return training(
        int(filter1), int(kernel_size1), int(pool_size1), 
        int(filter2), int(kernel_size2), int(pool_size2),
        int(filter3), int(kernel_size3), int(pool_size3),
        int(dense1), int(dense2), lr, int(epochs)
    )

In [22]:
pbounds = {
    'filter1': (32, 128), 'kernel_size1': (1,5), 'pool_size1': (1,5),
    'filter2': (64, 256), 'kernel_size2': (1,5), 'pool_size2': (1,5),
    'filter3': (128, 512), 'kernel_size3': (1,5), 'pool_size3': (1,5),
    'dense1': (256, 512), 'dense2': (128, 256), 
    'lr': (1e-4, 1e-2), 'epochs': (5, 15)
}

In [23]:
bo_optimizer = BayesianOptimization(
    f=None,
    pbounds=pbounds,
    verbose=2,
    random_state=123
)

In [24]:
utility = UtilityFunction(
    kind='ucb', # 탐색과 활용 사이의 균형 유지, 불확실한 지점을 더 많이 탐색
    kappa=2.5,  # UCB전략에서 얼마의 불확실성을 고려할지 지정
                # 값이 높으면 탐색을, 값이 낮으면 활용을 강조함
    xi=0.0      # 탐색전략에서 사용되는 파라미터로 얼마나 큰 개선을 고려할지 조정
                # 값이 높으면 큰 개선을 값이 낮으면 작은 개선을 탐색함
)

In [25]:
next_point = bo_optimizer.suggest(utility)
print(next_point)

{'dense1': 434.29611151305255, 'dense2': 164.62583487364856, 'epochs': 7.268514535642031, 'filter1': 84.92621783195756, 'filter2': 202.13804219882812, 'filter3': 290.472880687793, 'kernel_size1': 4.923056793538462, 'kernel_size2': 3.739318954339453, 'kernel_size3': 2.9237276059374437, 'lr': 0.0039819634301220905, 'pool_size1': 2.3727120646034776, 'pool_size2': 3.9161988295361665, 'pool_size3': 2.7542889787184976}


In [26]:
_start = datetime.now()
target = target_func(**next_point)
print(target)
print(f'처리시간: {datetime.now()-_start}')

2023-08-23 22:08:49.946115: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000,1]
	 [[{{node Placeholder/_1}}]]
2023-08-23 22:08:49.946375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000,1]
	 [[{{node Placeholder/_1}}]]
2023-08-23 22:08:50.990555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8904
2023-08-23 22:08:51.406234: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-23 22:08:51.767963: I tensorflow/compiler

0.5036
처리시간: 0:00:41.878946


In [27]:
bo_optimizer.register(params=next_point, target=target)

In [28]:
_start = datetime.now()
for loop in range(20):
    __start = datetime.now()
    next_point = bo_optimizer.suggest(utility)
    target = target_func(**next_point)
    bo_optimizer.register(params=next_point, target=target)
    print(
        f'idx({loop:>2}): {datetime.now()-__start}, '
        f'accuracy: {target}, params: {next_point}'
    )
print('--------------')
print(bo_optimizer.max)
print(f'처리시간: {datetime.now() - _start}')

ValueError: in user code:

    File "/tmp/ipykernel_7938/2119553101.py", line 4, in train_step  *
        predictions = model(images, training=True)
    File "/home/freeman/anaconda3/envs/t212p38/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_file_aysxdpj.py", line 10, in tf__call
        model = ag__.converted_call(ag__.ld(self).data, (ag__.ld(input),), None, fscope)

    ValueError: Exception encountered when calling layer 'Cifar10CNNModel' (type Cifar10CNNModel).
    
    in user code:
    
        File "/tmp/ipykernel_7938/2134668432.py", line 23, in call  *
            model = self.data(input)
        File "/home/freeman/anaconda3/envs/t212p38/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
            raise e.with_traceback(filtered_tb) from None
    
        ValueError: tf.function only supports singleto